In [3]:
import setup_django
setup_django.init()

In [27]:
import numpy as np 
import pandas as pd 
from django.contrib.auth import get_user_model
from django.contrib.contenttypes.models import ContentType
from django.db.models import F, Sum, Count, IntegerField
from movies.models import Movie
from ratings.models import Rating
import random

User = get_user_model()




In [ ]:
user = User.objects.create_user(
    username='omkar',
    email='omkar@example.com',
    password='test'
)

In [31]:
ctype = ContentType.objects.get(app_label='movies', model='movie')
movie = Movie.objects.get(title="Jumanji")
object_id = movie.id
rating_value = 4.0
rating_obj = Rating.objects.create(content_type=ctype, object_id=object_id, value=rating_value, user=user)

In [33]:
rating_obj = Rating.objects.filter(user__username='omkar', value__gte=3).first()
rating_obj.content_object

<Movie: Jumanji (1995)>

In [34]:
movie_id = rating_obj.content_object.id
value = rating_obj.value
user_id = rating_obj.user_id

In [35]:

print(f"Movie {movie_id} was rated {value} by {user_id}")

Movie 2 was rated 4 by 672


In [36]:
other_user_ids = Rating.objects.filter(
        object_id=rating_obj.object_id, 
        content_type=rating_obj.content_type, 
        value__gte=rating_obj.value
).exclude(user=rating_obj.user).values_list('user_id', flat=True)

highly_rated = Rating.objects.filter(user_id__in=other_user_ids, value__gte=4)

In [37]:
rec_users = []
rec_movies = []
for rating_obj in highly_rated:
    if rating_obj.user not in rec_users:
        rec_users.append(rating_obj.user)
    object_id = rating_obj.object_id
    if object_id not in rec_movies:
        rec_movies.append(object_id)

In [39]:
print(len(rec_users), len(rec_movies))

51 2815


In [40]:
import numpy as np

In [43]:
# Write a list of 2 movies attributes
movie_1 = ['sci-fi', 'comedy', 'Stars latest action movie star']
movie_2 = ['historical fiction', 'drama', 'Stars latest action movie star']

# Write a list of a User's preferences
user_a = ['comedy', 'sci-fi', 'Stars latest action movie star']
print(f"User has {len(user_a)} primary preferences")

# Total number of potential features
num_unique_features = len(set(movie_1 + movie_2))
print(f"These movies have {num_unique_features} unique features.")


User has 3 primary preferences
These movies have 5 unique features.


Convert the user preferences list into a numerical score between -1 and 1

In [44]:
my_preferences = np.array([.98, .85, .75])

Convert movie attributes into a numerical score between -1 and 1 based on the following 3 attributes:

* Sci-Fi
* Comedy
* Stars latest action hero

In [45]:
movie_attribute_scores = np.array([.96, .99, .92]) 

movie_2_attribute_scores = np.array([-.93, -.45, .5])

Predict User's likelihood of enjoying either movie using Matrix multiplication.

In [46]:
(my_preferences * movie_attribute_scores).sum() 

2.4723

In [47]:
(my_preferences * movie_2_attribute_scores).sum()

-0.9189

In [48]:
max_score = (np.array([1,1,1]) * np.array([1,1,1])).sum()
min_score = (np.array([1,1,1]) * np.array([-1,-1,-1])).sum()
print(max_score, min_score)

3 -3


In [49]:
ids = Movie.objects.all().popular()[:15]
most_active_users = Rating.objects.all().annotate(
    user_count=Count("user")
).order_by('-user_count').values_list('user_id')[:5_000]

ratings_qs =  Rating.objects.filter(
    object_id__in=ids, 
    user_id__in=most_active_users, 
    active=True, value__gte=0
).annotate(movie=F("object_id")).values('user', 'value', 'movie')

df = pd.DataFrame(ratings_qs)
df

,user,value,movie
0,26,5,1
1,24,5,6
2,25,3,3
3,23,3,1
4,23,4,6
5,23,4,11
6,21,3,10
7,20,4,1
8,19,3,1
9,19,3,2


In [50]:
df.pivot_table(index='user', columns=['movie'], values='value', fill_value='')[:15]

movie,1,2,3,4,5,6,7,9,10,11
user,,,,,,,,,,
2,,,,,,,,,4.0,
4,,,,,,,,,4.0,
5,,,4.0,,,,,,,
7,3.0,,,,,,,,3.0,
9,4.0,,,,,,,,,
13,5.0,,,,,,,,,
15,2.0,2.0,,,5.0,4.0,,,3.0,3.0
17,,,,,,5.0,,,,
18,,,,,3.0,4.0,3.0,3.0,,


In [51]:
movie_1 = np.array([1, 1, 1, 1, 1])
user_1 = np.array([1, 1, 1, 1, 1])
(user_1 * movie_1).sum()

5

<code> movie_1_features = np.array(["x", "y", "z", "xx", "yy"])
user_1_preferences = np.array(["x", "y", "z", "xx", "yy"])

assert (user_1_preferences * movie_1_features).sum() == 5 </code>

* These are called latent features. 

* They are hidden and it's what the machine learning model will try and find for us.